In [1]:
import requests
import zipfile
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import json

In [2]:
dataset_file_path = '/Users/kiranm/Downloads/DAEN 690/Datasets/Updated-Aggregates-Financing-2030-Agenda-2.csv'

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
df = pd.read_csv(dataset_file_path, header=0,  low_memory=False, encoding='utf-8').reset_index(drop=True)

In [4]:
directory_path = os.path.dirname(dataset_file_path)
print(directory_path)

C:\Users\siddh\OneDrive\Desktop\Project


In [5]:
print("\nMissing values per column:\n", df.isnull().sum())


Missing values per column:
 year                 0
donor_code           0
donor_name           0
donor_country        0
recipient_code       0
recipient_name       0
recipient_country    0
recipient_region     0
disbursement         0
total_proj           0
sdg_1_sum            0
sdg_1_n_proj         0
sdg_2_sum            0
sdg_2_n_proj         0
sdg_3_sum            0
sdg_3_n_proj         0
sdg_4_sum            0
sdg_4_n_proj         0
sdg_5_sum            0
sdg_5_n_proj         0
sdg_6_sum            0
sdg_6_n_proj         0
sdg_7_sum            0
sdg_7_n_proj         0
sdg_8_sum            0
sdg_8_n_proj         0
sdg_9_sum            0
sdg_9_n_proj         0
sdg_10_sum           0
sdg_10_n_proj        0
sdg_11_sum           0
sdg_11_n_proj        0
sdg_12_sum           0
sdg_12_n_proj        0
sdg_13_sum           0
sdg_13_n_proj        0
sdg_14_sum           0
sdg_14_n_proj        0
sdg_15_sum           0
sdg_15_n_proj        0
sdg_16_sum           0
sdg_16_n_proj        0
sdg_1

In [6]:
donor_code_map = df.groupby('donor_country')['donor_code'].first().to_dict()
recipient_code_map = df.groupby('recipient_country')['recipient_code'].first().to_dict()

df['donor_code'] = df['donor_country'].map(donor_code_map)
df['recipient_code'] = df['recipient_country'].map(recipient_code_map)

In [7]:
print("Original shape: ")
print(df.shape)
df = df.drop_duplicates()

Original shape: 
(82959, 48)


### GEO Coding

In [8]:
import pandas as pd
import pycountry
from geopy.geocoders import ArcGIS
from geopy.extra.rate_limiter import RateLimiter

In [9]:
def remove_third_element(point):
    """Removes the third element from a tuple if it exists."""
    if isinstance(point, tuple):
        return point[:2]
    else:
        return point

def get_code(name):
    """Retrieves the 2-letter country code for a given country name."""
    try:
        return pycountry.countries.lookup(name).alpha_2
    except LookupError:
        return None

geolocator = ArcGIS(user_agent="DAEN-690-Capstone")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_location(row, country_col, code_col):
    country_name = row[country_col]
    country_code = row[code_col]
    
    if country_code:
        return geocode(f"{country_name}, {country_code}")
    else:
        return geocode(country_name)

In [10]:
geonames_donor = df[['donor_code', 'donor_country']].drop_duplicates()
geonames_donor['Country Code'] = geonames_donor.donor_country.apply(get_code)
geonames_donor['Location'] = geonames_donor.apply(lambda row: get_location(row, 'donor_country', 'Country Code'), axis=1)
geonames_donor['Point'] = geonames_donor['Location'].apply(lambda loc: tuple(loc.point) if loc else None)

merged_df = df.merge(geonames_donor[['donor_country', 'Point']], on='donor_country', how='left')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

merged_df.Point = merged_df.Point.apply(remove_third_element)
merged_df[['donor_translate_y', 'donor_translate_x']] = merged_df.Point.apply(lambda x: pd.Series(x))

merged_df[['donor_lat', 'donor_long']] = merged_df[['donor_translate_y', 'donor_translate_x']]

In [11]:
geonames_recipient = df[['recipient_code', 'recipient_country']].drop_duplicates()
geonames_recipient['Recipient Country Code'] = geonames_recipient.recipient_country.apply(get_code)
geonames_recipient['Recipient Location'] = geonames_recipient.apply(lambda row: get_location(row, 'recipient_country', 'Recipient Country Code'), axis=1)
geonames_recipient['Recipient Point'] = geonames_recipient['Recipient Location'].apply(lambda loc: tuple(loc.point) if loc else None)

merged_df = merged_df.merge(geonames_recipient[['recipient_country', 'Recipient Point']], on='recipient_country', how='left')

merged_df['Recipient Point'] = merged_df['Recipient Point'].apply(remove_third_element)
merged_df[['recipient_translate_y', 'recipient_translate_x']] = merged_df['Recipient Point'].apply(lambda x: pd.Series(x))

merged_df[['recipient_lat', 'recipient_long']] = merged_df[['recipient_translate_y', 'recipient_translate_x']]
 
print(merged_df.head())

   year  donor_code donor_name donor_country  recipient_code  \
0  2010           1    Austria       Austria              55   
1  2010           1    Austria       Austria              57   
2  2010           1    Austria       Austria              62   
3  2010           1    Austria       Austria              63   
4  2010           1    Austria       Austria              64   

           recipient_name       recipient_country    recipient_region  \
0                 Turkiye                  Turkey  Europe and Eurasia   
1                  Kosovo                  Kosovo  Europe and Eurasia   
2                 Croatia                 Croatia  Europe and Eurasia   
3                  Serbia                  Serbia  Europe and Eurasia   
4  Bosnia and Herzegovina  Bosnia and Herzegovina  Europe and Eurasia   

   disbursement  total_proj  ...                   Point  donor_translate_y  \
0      28804294           7  ...  (47.90743, 13.8091438)           47.90743   
1      17303907   

In [12]:
print("Before: ", merged_df.shape)
merged_df = merged_df.drop_duplicates()
print("After: ", merged_df.shape)

Before:  (82959, 58)
After:  (82959, 58)


In [13]:
geo_path = os.path.join(directory_path, 'GeoLocationInfo.csv')
merged_df.to_csv(geo_path, index=False, encoding='utf-8')

### Drop pins

In [14]:
import pandas as pd
import os
import numpy as np

In [15]:
node_path = os.path.join(directory_path,"np_node-template.csv")
tag_path = os.path.join(directory_path,"np_tag-template.csv")
os.chmod(node_path, 0o777)

In [16]:
colors_path = os.path.join(directory_path, 'colors.csv')
colors = pd.read_csv(colors_path, index_col='np_color_id', encoding='utf-8')

In [17]:
temp_df = pd.read_csv(geo_path, encoding="utf-8")
temp_df = temp_df.sort_values(by=['donor_country', 'recipient_country',  'year'], ascending=[True, True,True])
donor_countries_list = temp_df["donor_country"].unique()
recipient_countries_list = temp_df["recipient_country"].unique()

In [18]:
def group_years_by_interval(df):
    def year_group(year):
        # Determine the 3-year interval
        start_year = (year - 2010) // 3 * 3 + 2010
        end_year = start_year + 2
        return f"{start_year}-{end_year}"
    
    df['year_interval'] = df['year'].apply(year_group)
    return df

def aggregate_data(df):
    sum_columns = [
        'disbursement', 'total_proj', 'sdg_1_sum', 'sdg_1_n_proj', 'sdg_2_sum', 'sdg_2_n_proj',
        'sdg_3_sum', 'sdg_3_n_proj', 'sdg_4_sum', 'sdg_4_n_proj', 'sdg_5_sum', 'sdg_5_n_proj',
        'sdg_6_sum', 'sdg_6_n_proj', 'sdg_7_sum', 'sdg_7_n_proj', 'sdg_8_sum', 'sdg_8_n_proj',
        'sdg_9_sum', 'sdg_9_n_proj', 'sdg_10_sum', 'sdg_10_n_proj', 'sdg_11_sum', 'sdg_11_n_proj',
        'sdg_12_sum', 'sdg_12_n_proj', 'sdg_13_sum', 'sdg_13_n_proj', 'sdg_14_sum', 'sdg_14_n_proj',
        'sdg_15_sum', 'sdg_15_n_proj', 'sdg_16_sum', 'sdg_16_n_proj', 'sdg_17_sum', 'sdg_17_n_proj',
        'env_sum', 'env_n_proj', 'non_sdg_sum', 'non_sdg_n_proj'
    ]

    aggregated_df = df.groupby(
        ['donor_country', 'recipient_country', 'year_interval'], as_index=False
    )[sum_columns].sum()

    other_columns = [
        'donor_name', 'recipient_name', 'donor_code', 'recipient_code',
        'donor_translate_y', 'donor_translate_x', 'donor_lat', 'donor_long',
        'recipient_translate_y', 'recipient_translate_x', 'recipient_lat', 'recipient_long', 'ring_location'
    ]
    
    result_df = pd.merge(
        aggregated_df,
        df.drop(columns=sum_columns).drop_duplicates(
            subset=['donor_country', 'recipient_country', 'year_interval']
        ),
        on=['donor_country', 'recipient_country', 'year_interval'],
        how='left'
    )
    return result_df

ring_location_dict = {
        "2010-2012": -135,
        "2013-2015": -100,
        "2016-2018": -65,
        "2019-2021": -30
    }
    
temp_df = group_years_by_interval(temp_df)
temp_df["ring_location"] = temp_df["year_interval"].map(ring_location_dict)
result_df = aggregate_data(temp_df)

donor_dict = {}
recipient_dict = {}

node_id = 1
for country in donor_countries_list:
    donor_dict[country] = node_id
    node_id+=1

for country in recipient_countries_list:
    recipient_dict[country] = node_id
    node_id+=1

result_df["donor_node_id"] = result_df["donor_country"].map(donor_dict)
result_df["recipient_node_id"] = result_df["recipient_country"].map(recipient_dict)

result_df = result_df[result_df['disbursement'] != 0]

In [19]:
columns_to_drop = [
    'year', 'donor_code', 'donor_name', 'recipient_code',
    'recipient_name', 'donor_translate_y', 'donor_translate_x',
    'recipient_translate_y', 'recipient_translate_x'
]

result_df = result_df.drop(columns=columns_to_drop, errors='ignore')

In [20]:
def normalize_value(x, a, b):
    if b - a == 0: 
        return 0.01
    return 0.01 + ((x - a) * (2 - 0.01)) / (b - a)

sdg_columns = [col for col in result_df.columns if '_sum' in col.lower()]

for col in sdg_columns:
    a = result_df[col].min()  
    b = result_df[col].max()  
    result_df[f'{col}_ratio'] = result_df[col].apply(lambda x: normalize_value(x, a, b))


In [21]:
result_df.columns

Index(['donor_country', 'recipient_country', 'year_interval', 'disbursement',
       'total_proj', 'sdg_1_sum', 'sdg_1_n_proj', 'sdg_2_sum', 'sdg_2_n_proj',
       'sdg_3_sum', 'sdg_3_n_proj', 'sdg_4_sum', 'sdg_4_n_proj', 'sdg_5_sum',
       'sdg_5_n_proj', 'sdg_6_sum', 'sdg_6_n_proj', 'sdg_7_sum',
       'sdg_7_n_proj', 'sdg_8_sum', 'sdg_8_n_proj', 'sdg_9_sum',
       'sdg_9_n_proj', 'sdg_10_sum', 'sdg_10_n_proj', 'sdg_11_sum',
       'sdg_11_n_proj', 'sdg_12_sum', 'sdg_12_n_proj', 'sdg_13_sum',
       'sdg_13_n_proj', 'sdg_14_sum', 'sdg_14_n_proj', 'sdg_15_sum',
       'sdg_15_n_proj', 'sdg_16_sum', 'sdg_16_n_proj', 'sdg_17_sum',
       'sdg_17_n_proj', 'env_sum', 'env_n_proj', 'non_sdg_sum',
       'non_sdg_n_proj', 'recipient_region', 'Point', 'donor_lat',
       'donor_long', 'Recipient Point', 'recipient_lat', 'recipient_long',
       'ring_location', 'donor_node_id', 'recipient_node_id',
       'sdg_1_sum_ratio', 'sdg_2_sum_ratio', 'sdg_3_sum_ratio',
       'sdg_4_sum_ratio', 's

In [22]:
node_df = pd.read_csv(node_path)
tag_df = pd.read_csv(tag_path)

def create_country_node_tag(node_id, lat, long, z, colors, title, description):
    global node_df, tag_df  
    
    new_pin = node_df.iloc[0].copy()
    new_tag = tag_df.iloc[0].copy()
    
    new_pin[['np_node_id', 'np_tag_id', 'record_id']] = node_id
    new_pin['np_table_id'] = 1
    new_pin['parent_id'] = 0 
    new_pin[['scale_x', 'scale_y', 'scale_z']] = 0.5
    new_pin['translate_x'] = long
    new_pin['translate_y'] = lat
    new_pin['translate_z'] = z
    new_pin['np_geometry_id'] = 19
    new_pin['np_topo_id'] = 6
    new_pin['np_color_id'] = 1
    new_pin[['color_r', 'color_g', 'color_b']] = colors

    new_tag[['np_tag_id', 'record_id']] = node_id
    new_tag['table_id'] = 1
    new_tag['title'] = title
    new_tag['description'] = description

    node_df = pd.concat([node_df, pd.DataFrame(new_pin).T], ignore_index=True)
    tag_df = pd.concat([tag_df, pd.DataFrame(new_tag).T], ignore_index=True)

def create_year_node_tag(ring_location, parent_id, ring_id, title):
    global node_df, tag_df
    new_year_ring = node_df.iloc[1].copy()
    new_year_tag = tag_df.iloc[0].copy()
    new_year_ring[['np_node_id', 'np_tag_id', 'record_id']] = ring_id
    new_year_ring['parent_id'] = parent_id
    new_year_ring['branch_level'] = 2
    new_year_ring['translate_x'] = ring_location
    new_year_ring[['scale_x', 'scale_y', 'scale_z']] = 0.5
    new_year_ring['np_table_id'] = 1
    new_year_ring['np_color_id'] = 20
    new_year_ring[['color_r', 'color_g', 'color_b']] = [55, 190, 190]
    new_year_tag[['np_tag_id', 'record_id']] = ring_id
    new_year_tag['table_id'] = 1
    new_year_tag['title'] = title
    new_year_tag['description'] = 'Year'
    node_df = pd.concat([node_df, pd.DataFrame(new_year_ring).T], ignore_index=True)
    tag_df = pd.concat([tag_df, pd.DataFrame(new_year_tag).T], ignore_index=True)

def create_petal_rings(petal_id, parent_id, location, title, colors):
    global node_df, tag_df
    new_series_ring = node_df.iloc[1].copy()
    new_series_tag = tag_df.iloc[0].copy()
    new_series_ring[["np_node_id", "np_data_id",  "np_tag_id", "record_id"]] = petal_id
    new_series_ring["parent_id"] = parent_id
    new_series_ring["branch_level"]=3
    new_series_ring["translate_x"] = location
    new_series_ring[['scale_x', 'scale_y', 'scale_z']] = 1
    new_series_ring['np_table_id'] = 1
    new_series_ring["np_topo_id"] = 3
    new_series_ring["ratio"] = 0.1
    new_series_ring[['scale_x','scale_y','scale_z']] = 1
    new_series_ring[['color_r', 'color_g', 'color_b']] = colors
    new_series_tag[["np_tag_id", "record_id"]] = petal_id
    new_series_tag["table_id"] = 1
    new_series_tag["title"] = title
    new_series_tag["description"] = "SDG"
    
    node_df = pd.concat([node_df, pd.DataFrame(new_series_ring).T], ignore_index=True)
    tag_df = pd.concat([tag_df, pd.DataFrame(new_series_tag).T], ignore_index=True)

def link_nodes(link_id, parent_id, child_id, title, colors, ratio):
    global node_df, tag_df
    new_pin = node_df.iloc[0].copy()
    new_tag = tag_df.iloc[0].copy()
    new_pin[["np_node_id", "np_tag_id", "record_id"]] = link_id
    new_pin['np_table_id'] = 1
    new_pin['np_geometry_id'] = 3
    new_pin['np_topo_id'] = 6
    new_pin['np_color_id'] = 20
    new_pin["ratio"] = ratio
    new_pin['parent_id'] = parent_id
    new_pin['child_id'] = child_id
    new_pin['type'] = 7
    new_pin[['color_r', 'color_g', 'color_b']] = colors
    new_tag[["np_tag_id", "record_id"]] = link_id
    new_tag['table_id'] = 1
    new_tag["title"] = title
    new_tag["description"] = title
    
    node_df = pd.concat([node_df, pd.DataFrame(new_pin).T], ignore_index=True)
    tag_df = pd.concat([tag_df, pd.DataFrame(new_tag).T], ignore_index=True)

In [23]:
import random
donor_countries_set = set()
recipient_countries_set = set()
donor_year_dict = {}
recipient_year_dict= {}
sdg_color_map = {
    'sdg_1': [255, 255, 255],  # SDG_1 = White
    'sdg_2': [0, 0, 255],      # SDG_2 = Blue
    'sdg_3': [255, 255, 0],    # SDG_3 = Yellow
    'sdg_4': [152, 0, 255],    # SDG_4 = Electric Violet
    'sdg_5': [255, 168, 0],    # SDG_5 = Orange
    'sdg_6': [0, 255, 255],    # SDG_6 = Cyan
    'sdg_7': [255, 0, 255],    # SDG_7 = Fuchsia (Magenta)
    'sdg_8': [0, 153, 0],      # SDG_8 = Forest Green
    'sdg_9': [185, 153, 102],  # SDG_9 = Lion (Gold)
    'sdg_10': [255, 180, 255], # SDG_10 = Mauve (Pink Pearl)
    'sdg_11': [0, 152, 255],   # SDG_11 = Dodgers Blue (Sky Blue)
    'sdg_12': [185, 255, 0],   # SDG_12 = Lime
    'sdg_13': [152, 0, 0],     # SDG_13 = Penn Red (Dark Red)
    'sdg_14': [127, 127, 127], # SDG_14 = Gray (Grey)
    'sdg_15': [127, 127, 255], # SDG_15 = Tropical Indigo (Lavender)
    'sdg_16': [197, 82, 0],    # SDG_16 = Tawny (Rust)
    'sdg_17': [0, 0, 0]        # SDG_17 = Black
}

ring_id = result_df["recipient_node_id"].max() + 1

for itr, row in result_df.iterrows():
    if row["donor_country"] not in donor_countries_set:
        create_country_node_tag(
            node_id=row["donor_node_id"],
            lat=row["donor_lat"],
            long=row["donor_long"],
            z=20,
            colors=[0, 225, 0],
            title=row["donor_country"],
            description="Donor Country"
        )
        donor_countries_set.add(row["donor_country"])
    
    if row["donor_country"] not in donor_year_dict:
        donor_year_dict[row["donor_country"]] = {}
    
    if row["recipient_country"] not in recipient_countries_set:
        create_country_node_tag(
            node_id=row["recipient_node_id"],
            lat=row["recipient_lat"],
            long=row["recipient_long"],
            z=0,
            colors=[225, 0, 0],
            title=row["recipient_country"],
            description="Recipient Country"
        )
        recipient_countries_set.add(row["recipient_country"])

    if row["recipient_country"] not in recipient_year_dict:
        recipient_year_dict[row["recipient_country"]] = {}
    
    if row['year_interval'] not in donor_year_dict[row["donor_country"]]:
        create_year_node_tag(
            ring_location=row["ring_location"],
            parent_id=row["donor_node_id"],
            ring_id=ring_id,
            title=f"{row['year_interval']} (Donor)"
        )
        petal_id = ring_id
        location = -45
        for sdg in sdg_columns:
            if int(row[sdg]) > 0:
                petal_id+=1 
                location+=30
                sdg_key = sdg.lower().replace('_sum', '')
                petal_color = sdg_color_map.get(sdg_key, [0, 0, 0])
                create_petal_rings(
                    petal_id=petal_id,
                    parent_id=ring_id,
                    location=location,
                    title=f"{sdg.upper()}", 
                    colors=petal_color
                )
                sdg_dict = {sdg: petal_id}

                if row["year_interval"] not in donor_year_dict[row["donor_country"]]:
                    donor_year_dict[row["donor_country"]][row["year_interval"]] = []

                donor_year_dict[row["donor_country"]][row["year_interval"]].append(sdg_dict)
        ring_id = petal_id + 1

    if row['year_interval'] not in recipient_year_dict[row["recipient_country"]]:
        create_year_node_tag(
            ring_location=row["ring_location"],
            parent_id=row["recipient_node_id"],
            ring_id=ring_id,
            title=f"{row['year_interval']} (Recipient)"
        )
        
        petal_id = ring_id
        location = -45
        for sdg in sdg_columns:
            if int(row[sdg]) > 0:
                petal_id+=1
                location+=30
                sdg_key = sdg.lower().replace('_sum', '')
                petal_color=sdg_color_map.get(sdg_key, [0, 0, 0])
                create_petal_rings(
                    petal_id=petal_id,
                    parent_id=ring_id,
                    location=location,
                    title=f"{sdg.upper()}",
                    colors=petal_color
                )
                sdg_dict = {sdg: petal_id}

                if row["year_interval"] not in recipient_year_dict[row["recipient_country"]]:
                    recipient_year_dict[row["recipient_country"]][row["year_interval"]] = []

                recipient_year_dict[row["recipient_country"]][row["year_interval"]].append(sdg_dict)
        ring_id = petal_id + 1

parent_child_mapping = []

import time

link_id = ring_id+1 
for itr, row in result_df.iterrows():
    if donor_year_dict[row["donor_country"]]:
        donor_country_data_list =  donor_year_dict[row["donor_country"]][row["year_interval"]]
        recipient_country_data_list =  recipient_year_dict[row["recipient_country"]][row["year_interval"]]
        for donor in donor_country_data_list:
            for key, parent_id in donor.items(): 
                for recipient in recipient_country_data_list:
                    if key in recipient:
                        child_id = recipient[key]
                        sdg_value = row[key]
                        if int(sdg_value) > 0:
                            sdg_key = key.lower().replace('_sum', '')
                            link_color = sdg_color_map.get(sdg_key, [0, 0, 0])
                            ratio_var = key + "_ratio"
                            ratio_value = row[ratio_var]
                            link_nodes(
                                link_id=link_id,
                                parent_id=parent_id,
                                child_id=child_id, 
                                title=f"{row['donor_country']} to {row['recipient_country']}: {key.upper()} {row[key]}",
                                ratio=ratio_value,
                                colors=link_color
                            )
                            link_id+=1

In [24]:
node_df = node_df.iloc[2:]
tag_df = tag_df.iloc[1:]

node_ints = ['np_node_id','type','np_data_id','selected','parent_id','branch_level','child_id','np_tag_id','np_palette_id','np_ch_in_id','np_ch_out_id','ch_sync_time',
             'np_palette_id_alt','np_color_id_alt', 'np_material_id','np_geometry_id', 'np_color_id', 'color_fade','np_texture_id','hide','freeze','np_topo_id',
             'subspace','trigger_hi_x','trigger_hi_y','trigger_hi_z','trigger_lo_x','trigger_lo_y','trigger_lo_z', 'proximity_x','proximity_y','proximity_z',
             'proximity_mode_x','proximity_mode_y','proximity_mode_z','segments_x','segments_y','segments_z','tag_mode','np_format_id','np_table_id','size']

node_df[node_ints] = node_df[node_ints].astype(int)
node_df['record_id'] = node_df['record_id'].astype(np.int64)
node_df.to_csv("/Users/kiranm/Downloads/DAEN 690/Datasets/proto-20240417T093505_np_node.csv", mode='w', index=False, encoding='utf-8', lineterminator='\r\n')

tag_df[["np_tag_id", "record_id"]] = tag_df[["np_tag_id", "record_id"]].astype(int)
tag_df.to_csv("/Users/kiranm/Downloads/DAEN 690/Datasets/proto-20240417T093505_np_tag.csv", mode='w', index=False, encoding='utf-8', lineterminator='\r\n')